*Note:** This guide requires **Databricks SQL, Databricks Runtime 17.0 or above**, and Unity Catalog only.

# Introduction
This guide shows how to design a Databricks medallion architecture (bronze, silver, gold) pipeline with SQL, using sample tables and realistic transformations relevant to a **healthcare payer**. All code is written to work in Databricks SQL notebooks.


# What is a lakehouse?

1. **Hybrid Architecture:**  
   A lakehouse combines the best of data lakes (flexible, cheap storage) and data warehouses (structured, fast analytics), providing transactional and governance features on top of open cloud storage.

2. **ACID Transactions and Schema Governance:**  
   Lakehouses support ACID transactions for consistent concurrent data access and enforce schema management, which is essential for data integrity and compliance.

3. **Open and Decoupled:**  
   They use open file formats (like Parquet), decouple compute from storage for flexible scalability, and allow access by a variety of analytics, BI, and machine learning tools.

4. **Supports All Workloads and Data Types:**  
   The architecture enables SQL analytics, data science, machine learning, and can handle structured, semi-structured, and unstructured data (including images, text, video).

5. **Single Platform, Enterprise Ready:**  
   With features like real-time streaming, end-to-end governance, access control, and data discovery tools, lakehouses reduce complexity—allowing enterprises to manage all data and analytics needs in one unified system.
![](https://www.databricks.com/wp-content/uploads/2020/01/data-lakehouse-new.png)

## Unity Catalog

[Unified and open governance for data and AI in the Lakehouse](https://www.databricks.com/product/unity-catalog#features)

[Demo link UC](https://adb-984752964297111.11.azuredatabricks.net/explore/data/quickstart_catalog_vkm_external?o=984752964297111&filterString=payorayor)

[Demo link (Discover Your org's Trusted Data & AI Assets)](https://adb-984752964297111.11.azuredatabricks.net/discover?o=984752964297111)

Eliminate silos, simplify governance and accelerate insights at scale

- Centralizes governance, access control, auditing, and data discovery for all data and AI assets across Databricks workspaces.
- Enables fine-grained, consistent data access policies (row- and column-level), defined once and applied everywhere.
- Provides comprehensive data lineage and audit logs, showing how and by whom data is accessed and transformed.
- Supports data discovery, tagging, and documentation, making it easier to find and understand datasets and models.
- Works across multiple clouds and supports open formats (Delta, Parquet, etc.), avoiding vendor lock-in and enabling broad interoperability.
- Allows secure data and AI sharing within and outside the organization, including clean rooms and partner collaborations.
- Provides built-in monitoring for data quality, freshness, and usage, helping ensure compliance and rapid troubleshooting.
- Integrates tightly with the catalog/schema/object model, enhancing organization and security for all managed data assets.

![](https://www.databricks.com/sites/default/files/2025-05/header-unity-catalog.png?v=1748513086)

[Unity Catalog Search & Data Explorer](https://app.getreprise.com/launch/96mpAqy/)

[Exploring Lineage and Governance with Unity Catalog](https://app.getreprise.com/launch/MnqjQDX/)

[A Comprehensive Guide to Data and AI Governance](https://www.databricks.com/sites/default/files/2024-08/comprehensive-guide-to-data-and-ai-governance.pdf)






## Medallion lakehouse architecture

In this example, we will be following the **medallion lakehouse architecture**. The medallion architecture is a data design pattern to organize data in a lakehouse. The goal is to progressively improve the quality and structure of the data as it flows through each layer (Bronze [**raw**] → Silver [**staging**] → Gold [**main**]).

1. **Bronze layer**: the raw, unvalidated data
2. **Silver**: cleansed and conformed data
3. **Gold**: curated business-level tables

<img src="https://www.databricks.com/sites/default/files/inline-images/building-data-pipelines-with-delta-lake-120823.png?v=1702318922" alt="Managed Tables" width="600" height="500">

## Managed tables

[Demo Link](https://adb-984752964297111.11.azuredatabricks.net/explore/data/quickstart_catalog_vkm_external/payor_bronze/claims_raw?o=984752964297111&activeTab=details)

[How Unity Catalog Managed Tables Automate Performance at Scale](https://www.databricks.com/blog/how-unity-catalog-managed-tables-automate-performance-scale) with [Predictive Optimization](https://www.databricks.com/blog/predictive-optimization-automatically-delivers-faster-queries-and-lower-tco)


<!-- ![](https://www.databricks.com/sites/default/files/inline-images/image2_48.png?v=1751297384) -->

<img src="https://www.databricks.com/sites/default/files/inline-images/image2_48.png?v=1751297384" alt="Managed Tables" width="600" height="500">


[Faster Queries: 20X query latency reduction](https://www.databricks.com/blog/predictive-optimization-automatically-delivers-faster-queries-and-lower-tco)

[Demo Link](https://adb-984752964297111.11.azuredatabricks.net/explore/data/quickstart_catalog_vkm_external?o=984752964297111&activeTab=details)

**Predictive Optimization** in Databricks automates table management by leveraging Unity Catalog and the Data Intelligence Platform. This innovative feature currently runs the following optimizations for Unity Catalog managed tables:

<img src="https://www.databricks.com/sites/default/files/styles/max_1000x1000/public/2024-05/db-976-blog-img-og.png?itok=qWBT8VA-&v=1717158571" alt="Managed Tables" width="600" height="500">

**Compaction** - This enhances query performance by optimizing file sizes, ensuring that data retrieval is efficient.

**Liquid Clustering** - This technique incrementally clusters incoming data, enabling optimal data layout and efficient data skipping.

**VACUUM** - This operation helps reduce costs by deleting unneeded files from storage.

**ANALYZE** - Predictive Optimization will intelligently update statistics used to optimize query plans, by running ANALYZE in the background. 



# Databricks Medallion Pipeline for a Healthcare Payer


## Modeling Concepts

Databricks fully supports both **dimensional modeling** (Kimball/star schema) and **Inmon-style, Data Vault architectures (hubs, satellites, links)** on the Lakehouse platform. For dimensional models, you can build classic star and snowflake schemas directly with SQL, benefiting from ACID transactions and scalable Delta Lake tables.

For Inmon/Data Vault use cases, Databricks provides rich support for hub-and-satellite models that address core enterprise needs for history, auditability, and extensibility—find end.

The Lakehouse approach lets you mix these styles as needed within a single platform, so you can incrementally land data in Raw Vault/EDW structures and later expose it as dimensional marts—all with Delta Live Tables, fine-grained security, and open formats.

Key blog resources:

[Dimensional Modeling](https://www.databricks.com/blog/implementing-dimensional-data-warehouse-databricks-sql-part-1)

[Data Vault/Hub-Satellite](https://www.databricks.com/blog/2022/06/24/prescriptive-guidance-for-implementing-a-data-vault-model-on-the-databricks-lakehouse-platform.html) 

[Data Vault best practices](https://www.databricks.com/blog/data-vault-best-practice-implementation-lakehouse)

<div style="display: flex; justify-content: space-between;">
  <img src="https://user-gen-media-assets.s3.amazonaws.com/gpt4o_images/5c87faea-3e60-4f71-826d-42d04f6cdc0b.png" alt="Managed Tables" width="400" height="350">
  <img src="https://user-gen-media-assets.s3.amazonaws.com/gpt4o_images/6826c275-d462-4c07-a978-43fe9c40f3ed.png" alt="Managed Tables" width="400" height="350">
</div>






## Sample Data Model

For a payer, commonly used tables include:

- **Members**: members enrolled in a health plan

- **Claims**: medical claim submissions

- **Providers**: healthcare providers (doctors, clinics)

- **Diagnoses**: claim diagnosis codes

- **Procedures**: procedures/services performed

Each table should have at least 50 rows.

<img src="https://user-gen-media-assets.s3.amazonaws.com/gpt4o_images/bdd54dc0-f3c7-4975-80a3-0017ebdb121c.png" alt="Managed Tables" width="400" height="300">





## Table	Key Columns

**Members**	member_id, first_name, last_name, birth_date, gender, plan_id, effective_date

**Claims**	claim_id, member_id, provider_id, claim_date, total_charge, claim_status

**Providers**	provider_id, npi, provider_name, specialty, address, city, state

**Diagnoses**	claim_id, diagnosis_code, diagnosis_desc

**Procedures**	claim_id, procedure_code, procedure_desc, amount

# Setup

In [0]:
dbutils.widgets.text("catalog", "quickstart_catalog_vkm_external", "Catalog")
dbutils.widgets.text("bronze_db", "payor_bronze", "Bronze DB")
dbutils.widgets.text("silver_db", "payor_silver", "Silver DB")
dbutils.widgets.text("gold_db", "payor_gold", "Gold DB")

catalog = dbutils.widgets.get("catalog")
bronze_db = dbutils.widgets.get("bronze_db")
silver_db = dbutils.widgets.get("silver_db")
gold_db = dbutils.widgets.get("gold_db")

path = f"/Volumes/{catalog}/{bronze_db}/payor/files/"

In [0]:
print(f"Catalog: {catalog}")
print(f"Bronze DB: {bronze_db}")
print(f"Silver DB: {silver_db}")
print(f"Gold DB: {gold_db}")
print(f"Path: {path}")

In [0]:
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")

In [0]:
spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"CREATE DATABASE IF NOT EXISTS {bronze_db}")
spark.sql(f"CREATE DATABASE IF NOT EXISTS {silver_db}")
spark.sql(f"CREATE DATABASE IF NOT EXISTS {gold_db}")

In [0]:
spark.sql(f"CREATE VOLUME IF NOT EXISTS {bronze_db}.payor")

In [0]:
# Create the volume and folders
dbutils.fs.mkdirs(f"/Volumes/{catalog}/{bronze_db}/payor/files/claims")
dbutils.fs.mkdirs(f"/Volumes/{catalog}/{bronze_db}/payor/files/diagnosis")
dbutils.fs.mkdirs(f"/Volumes/{catalog}/{bronze_db}/payor/files/procedures")
dbutils.fs.mkdirs(f"/Volumes/{catalog}/{bronze_db}/payor/files/members")
dbutils.fs.mkdirs(f"/Volumes/{catalog}/{bronze_db}/payor/files/providers")
dbutils.fs.mkdirs(f"/Volumes/{catalog}/{bronze_db}/payor/downloads")

In [0]:
import requests
import zipfile
import io
import os
import shutil

# Define the URL of the ZIP file
# url = "https://github.com/bigdatavik/notebookassets/blob/6ca9ed60c60e37b6e98d90cb7817746e7880e170/common/Payor_Archive.zip?raw=true"

url = "https://github.com/bigdatavik/databricksfirststeps/blob/6b225621c3c010a2734ab604efd79c15ec6c71b8/data/Payor_Archive.zip?raw=true"


# Download the ZIP file
response = requests.get(url)
zip_file = zipfile.ZipFile(io.BytesIO(response.content))

# Define the base path
base_path = f"/Volumes/{catalog}/{bronze_db}/payor/downloads" 

# Extract the ZIP file to the base path
zip_file.extractall(base_path)

# Define the paths
paths = {
    "claims.csv": f"{base_path}/claims",
    "diagnoses.csv": f"{base_path}/diagnosis",
    "procedures.csv": f"{base_path}/procedures",
    "member.csv": f"{base_path}/members",
    "providers.csv": f"{base_path}/providers"
}

# Create the destination directories if they do not exist
for dest_path in paths.values():
    os.makedirs(dest_path, exist_ok=True)

# Move the files to the respective directories
for file_name, dest_path in paths.items():
    source_file = f"{base_path}/{file_name}"
    if os.path.exists(source_file):
        os.rename(source_file, f"{dest_path}/{file_name}")


In [0]:
%python
# Copy the files to the specified directories and print the paths
shutil.copy(f"{base_path}/claims/claims.csv", f"/Volumes/{catalog}/{bronze_db}/payor/files/claims/claims.csv")
print(f"Copied to /Volumes/{catalog}/{bronze_db}/payor/files/claims/claims.csv")

shutil.copy(f"{base_path}/diagnosis/diagnoses.csv", f"/Volumes/{catalog}/{bronze_db}/payor/files/diagnosis/diagnosis.csv")
print(f"Copied to /Volumes/{catalog}/{bronze_db}/payor/files/diagnosis/diagnosis.csv")

shutil.copy(f"{base_path}/procedures/procedures.csv", f"/Volumes/{catalog}/{bronze_db}/payor/files/procedures/procedures.csv")
print(f"Copied to /Volumes/{catalog}/{bronze_db}/payor/files/procedures/procedures.csv")

shutil.copy(f"{base_path}/members/member.csv", f"/Volumes/{catalog}/{bronze_db}/payor/files/members/members.csv")
print(f"Copied to /Volumes/{catalog}/{bronze_db}/payor/files/members/members.csv")

shutil.copy(f"{base_path}/providers/providers.csv", f"/Volumes/{catalog}/{bronze_db}/payor/files/providers/providers.csv")
print(f"Copied to /Volumes/{catalog}/{bronze_db}/payor/files/providers/providers.csv")

[Upload files](https://github.com/bigdatavik/notebookassets/blob/6ca9ed60c60e37b6e98d90cb7817746e7880e170/common/Payor_Archive.zip) to your Volumes or directly to ADLS on Azure, e.g.:

members.csv

claims.csv

providers.csv

diagnoses.csv

procedures.csv


# Bronze Layer – Ingest Raw Data

The bronze layer ingests raw files (CSV, JSON, Parquet) and lands them in Delta tables with minimal transformation.

**Example: Create Bronze Tables**

In [0]:
# %sql
# LIST '/Volumes/quickstart_catalog_vkm_external/payor_bronze/payor/files/'

## COPY INTO

[COPY INTO](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/delta-copy-into) Loads data from a file location into a Delta table. This is a retryable and idempotent operation — Files in the source location that have already been loaded are skipped.

[Examples](https://learn.microsoft.com/en-us/azure/databricks/ingestion/cloud-object-storage/copy-into/)

[Tutorial](https://learn.microsoft.com/en-us/azure/databricks/ingestion/cloud-object-storage/copy-into/tutorial-notebook)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {bronze_db}.claims_raw (
    claim_id STRING,
    member_id STRING,
    provider_id STRING,
    claim_date STRING,
    total_charge STRING,
    claim_status STRING
)
""")

In [0]:
copy_into_query = f"""
COPY INTO {bronze_db}.claims_raw
FROM '{path}/claims/'
FILEFORMAT = CSV
FORMAT_OPTIONS (
    'header' = 'true'
)
"""
spark.sql(copy_into_query)


In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {bronze_db}.diagnosis_raw (
    claim_id STRING,
    diagnosis_code STRING,
    diagnosis_desc STRING
)
""")

In [0]:
copy_into_diag = f"""
COPY INTO {bronze_db}.diagnosis_raw
FROM '{path}/diagnosis/'
FILEFORMAT = CSV
FORMAT_OPTIONS (
    'header' = 'true'
)
"""
spark.sql(copy_into_diag)


In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {bronze_db}.members_raw (
    member_id STRING,
    first_name STRING,
    last_name STRING,
    birth_date STRING,
    gender STRING,
    plan_id STRING,
    effective_date STRING
)
""")

In [0]:
copy_into_members = f"""
COPY INTO {bronze_db}.members_raw
FROM '{path}/members'
FILEFORMAT = CSV
FORMAT_OPTIONS (
    'header' = 'true'
)
"""
spark.sql(copy_into_members)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {bronze_db}.procedures_raw (
    claim_id STRING,
    procedure_code STRING,
    procedure_desc STRING,
    amount STRING
)
""")


In [0]:
copy_into_procedures = f"""
COPY INTO {bronze_db}.procedures_raw
FROM '{path}/procedures'
FILEFORMAT = CSV
FORMAT_OPTIONS (
    'header' = 'true'
)
"""
spark.sql(copy_into_procedures)


In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {bronze_db}.providers_raw (
    provider_id STRING,
    npi STRING,
    provider_name STRING,
    specialty STRING,
    address STRING,
    city STRING,
    state STRING
)
""")


In [0]:
copy_into_providers = f"""
COPY INTO {bronze_db}.providers_raw
FROM '{path}/providers'
FILEFORMAT = CSV
FORMAT_OPTIONS (
    'header' = 'true'
)
"""
spark.sql(copy_into_providers)


# Silver Layer – Transform, Clean, and Join

The silver layer cleans data, applies business rules, type-casts fields, deduplicates, and joins tables when needed.

**Example: Transform Bronze to Silver**

Create Silver Schema and Deduplicate Data

In [0]:
# Create ETL log table
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {silver_db}.etl_log (
  etl_timestamp TIMESTAMP,
  operation STRING,
  details STRING
)
""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE PROCEDURE {silver_db}.sprocs_transform_members()
LANGUAGE SQL
SQL SECURITY INVOKER
AS
BEGIN
    DECLARE run_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP();
    CREATE OR REPLACE TABLE {silver_db}.members AS
    SELECT DISTINCT
        CAST(member_id AS STRING) AS member_id,
        TRIM(first_name) AS first_name,
        TRIM(last_name) AS last_name,
        CAST(birth_date AS DATE) AS birth_date,
        gender,
        plan_id,
        CAST(effective_date AS DATE) AS effective_date
    FROM {bronze_db}.members_raw
    WHERE member_id IS NOT NULL;

     -- Log the ETL operation
    INSERT INTO {silver_db}.etl_log (etl_timestamp, operation, details)
    VALUES (
        run_timestamp,
        'sprocs_transform_members_with_log',
        'Loaded members from bronze to silver'
    );
END
""")

In [0]:
# Command to call the procedure
spark.sql(f"CALL {silver_db}.sprocs_transform_members()")

In [0]:
spark.sql(f"""
CREATE OR REPLACE PROCEDURE {silver_db}.create_clean_claims_table()
LANGUAGE SQL
SQL SECURITY INVOKER
AS
BEGIN
  DECLARE run_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP();
  CREATE OR REPLACE TABLE {silver_db}.claims AS
  SELECT DISTINCT
    claim_id,
    member_id,
    provider_id,
    CAST(claim_date AS DATE) AS claim_date,
    ROUND(try_cast(total_charge AS DECIMAL(10, 2)), 2) AS total_charge,
    LOWER(claim_status) AS claim_status
  FROM {bronze_db}.claims_raw
  WHERE claim_id IS NOT NULL AND try_cast(total_charge AS DECIMAL(10, 2)) IS NOT NULL;

  -- Log the ETL operation
INSERT INTO {silver_db}.etl_log (etl_timestamp, operation, details)
VALUES (
    CURRENT_TIMESTAMP(),
    'create_clean_claims_table',
    'Loaded claims from bronze to silver with valid charges'
);
END;
""")

In [0]:
# Command to call the procedure
spark.sql(f"CALL {silver_db}.create_clean_claims_table()")

In [0]:
spark.sql(f"""
CREATE OR REPLACE PROCEDURE {silver_db}.create_providers_table()
LANGUAGE SQL
SQL SECURITY INVOKER
AS
BEGIN
  DECLARE run_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP();
  
  CREATE OR REPLACE TABLE {silver_db}.providers AS
  SELECT DISTINCT
    provider_id,
    npi,
    provider_name,
    specialty,
    address,
    city,
    state
  FROM {bronze_db}.providers_raw
  WHERE provider_id IS NOT NULL;

  -- Log the ETL operation for providers
  INSERT INTO {silver_db}.etl_log (etl_timestamp, operation, details)
  VALUES (
    CURRENT_TIMESTAMP(),
    'create_providers_table',
    'Loaded distinct providers from bronze to silver'
  );

END;
""")

In [0]:
# Command to call the procedure
spark.sql(f"CALL {silver_db}.create_providers_table()")

In [0]:
%python
spark.sql(f"""
CREATE OR REPLACE PROCEDURE {silver_db}.create_procedures_table()
LANGUAGE SQL
SQL SECURITY INVOKER
AS
BEGIN
  DECLARE run_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP();
  CREATE OR REPLACE TABLE {silver_db}.procedures AS
  SELECT DISTINCT
    claim_id,
    UPPER(procedure_code) AS procedure_code,
    procedure_desc,
    CAST(REPLACE(amount, '$', '') AS DOUBLE) AS amount
  FROM {bronze_db}.procedures_raw
  WHERE claim_id IS NOT NULL AND procedure_code IS NOT NULL;

-- Log the ETL operation for procedures
INSERT INTO {silver_db}.etl_log (etl_timestamp, operation, details)
VALUES (
    CURRENT_TIMESTAMP(),
    'create_procedures_table',
    'Loaded distinct procedures from bronze to silver'
);

END;
""")


In [0]:
# Command to call the procedure
spark.sql(f"CALL {silver_db}.create_procedures_table()")

In [0]:
spark.sql(f"""
CREATE OR REPLACE PROCEDURE {silver_db}.transform_diagnoses()
LANGUAGE SQL
SQL SECURITY INVOKER
AS
BEGIN
  DECLARE run_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP();
  CREATE OR REPLACE TABLE {silver_db}.diagnoses
  SELECT DISTINCT
    claim_id,
    UPPER(diagnosis_code) AS diagnosis_code,
    TRIM(diagnosis_desc) AS diagnosis_desc
  FROM {bronze_db}.diagnosis_raw
  WHERE claim_id IS NOT NULL AND diagnosis_code IS NOT NULL;

-- Log the ETL operation for diagnoses
INSERT INTO {silver_db}.etl_log (etl_timestamp, operation, details)
VALUES (
    CURRENT_TIMESTAMP(),
    'transform_diagnoses',
    'Loaded distinct diagnoses from bronze to silver'
);

END;
""")

In [0]:
# Command to call the procedure
spark.sql(f"CALL {silver_db}.transform_diagnoses()")


# Gold Layer – Aggregate, Model, Ready for Analytics

Gold tables are optimized for business usage: facts, dimensions, and aggregated views.

**Example: Build Analytics-Friendly Gold Tables**

Create Enriched Claims and Member Summary Tables

In [0]:
%python
# Create ETL log table in gold schema
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {gold_db}.etl_log (
  etl_timestamp TIMESTAMP,
  operation STRING,
  details STRING
)
""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE PROCEDURE {gold_db}.enrich_claims()
LANGUAGE SQL
SQL SECURITY INVOKER
AS
BEGIN
  DECLARE run_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP();
  CREATE OR REPLACE TABLE {gold_db}.claims_enriched AS
  SELECT
    c.claim_id,
    c.claim_date,
    c.total_charge,
    c.claim_status,
    m.member_id,
    m.first_name,
    m.last_name,
    m.gender,
    m.plan_id,
    p.provider_id,
    p.provider_name,
    p.specialty,
    p.city,
    p.state,
    d.diagnosis_code,
    d.diagnosis_desc
  FROM {silver_db}.claims c
  INNER JOIN {silver_db}.members m ON c.member_id = m.member_id
  INNER JOIN {silver_db}.providers p ON c.provider_id = p.provider_id
  LEFT JOIN {silver_db}.diagnoses d ON c.claim_id = d.claim_id;

-- ETL log insert for enrich_claims operation

INSERT INTO {gold_db}.etl_log (etl_timestamp, operation, details)
VALUES (
    CURRENT_TIMESTAMP(),
    'enrich_claims',
    'Loaded claims enrichment table in gold schema'
);
  
END;
""")

In [0]:
# Command to call the procedure
spark.sql(f"CALL {gold_db}.enrich_claims()")

In [0]:
%python
# Stored procedure to create member claim summary, with parameterized schemas
spark.sql(f"""
CREATE OR REPLACE PROCEDURE {gold_db}.create_member_claim_summary()
LANGUAGE SQL
SQL SECURITY INVOKER
AS
BEGIN
  CREATE OR REPLACE TABLE {gold_db}.member_claim_summary AS
  SELECT
    member_id,
    COUNT(DISTINCT claim_id) AS total_claims,
    SUM(total_charge) AS sum_claims,
    MAX(total_charge) AS max_claim,
    MIN(total_charge) AS min_claim
  FROM {silver_db}.claims
  GROUP BY member_id;

  INSERT INTO {gold_db}.etl_log (etl_timestamp, operation, details)
VALUES (
    CURRENT_TIMESTAMP(),
    'create_member_claim_summary',
    'Loaded member claim summary table in gold schema'
);
END;
""")

In [0]:
# Command to call the procedure
spark.sql(f"CALL {gold_db}.create_member_claim_summary()")

In [0]:
df = spark.table(f"{gold_db}.claims_enriched")
result = df.groupBy("claim_status").sum("total_charge")
display(result)

In [0]:
%python
spark.sql(
    f"""
    SELECT claim_status, SUM(total_charge) AS total_charge_sum
    FROM {gold_db}.claims_enriched
    GROUP BY claim_status
    """
)

In [0]:
%python
df = spark.table(f"{gold_db}.claims_enriched")
result = df.groupBy("gender").count()
display(result)

Databricks visualization. Run in Databricks to view.

In [0]:
display(spark.table(f"{gold_db}.claims_enriched").groupBy("claim_date").sum("total_charge").orderBy("claim_date"))

Databricks visualization. Run in Databricks to view.

In [0]:
display(spark.table(f"{gold_db}.claims_enriched").groupBy("city").count())

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
display(spark.table(f"{gold_db}.claims_enriched").select("total_charge"))

Databricks visualization. Run in Databricks to view.

In [0]:
display(spark.table(f"{gold_db}.claims_enriched").select("claim_date", "total_charge"))

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

# AI/BI

Intelligent analytics for everyone

Databricks AI/BI is a new type of business intelligence product designed to provide a deep understanding of your data's semantics, enabling self-service data analysis for everyone in your organization. AI/BI is built on a compound AI system that draws insights from the full lifecycle of your data across the Databricks platform, including ETL pipelines, lineage, and other queries.

<img src="https://www.databricks.com/sites/default/files/2025-05/hero-image-ai-bi-v2-2x.png?v=1748417271" alt="Managed Tables" width="600" height="500">

## Payer AI/BI Dashboard

[AI/BI Dashboard ](https://adb-984752964297111.11.azuredatabricks.net/dashboardsv3/01f06a2e1390178b9800d808358c49ad/published?o=984752964297111)

-- Show how to ask Genie followup questions on Dashboard


# Genie

Talk with your data

Now everyone can get insights from data simply by asking questions in natural language.

<img src="https://www.databricks.com/sites/default/files/2025-06/ai-bi-genie-hero.png?v=1749162682" alt="Managed Tables" width="600" height="500">



## Payer Genie Room
[Payer Genie Room](https://adb-984752964297111.11.azuredatabricks.net/genie/rooms/01f06a3068a81406a386e8eaefc74545?o=984752964297111)

